# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4ddda0dbce-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [43]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [45]:
# Checking the Director Quentin Tarantino on Q3772
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3772 ?p ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'Mira Sorvino')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', '48th British Academy Film Awards')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Edgar Awards')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'Academy Award for Best Director')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Hungarian Order of Merit')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'Lentapedia')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', "Palme d'Or")]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'star on Hollywood Walk of Fame')]
[('p', 'http://www.wikidata.org/prop/

In [60]:

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q3772 ?p ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q212026'), ('name', 'Mira Sorvino')]
[('object', 'http://www.wikidata.org/entity/Q2566554'), ('name', '48th British Academy Film Awards')]
[('object', 'http://www.wikidata.org/entity/Q833154'), ('name', 'Edgar Awards')]
[('object', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('object', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('object', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('object', 'http://www.wikidata.org/entity/Q113068'), ('name', 'Hungarian Order of Merit')]
[('object', 'http://www.wikidata.org/entity/Q13452531'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('object', 'http://www.wikidata.org/entity/Q17290934'), ('name', 'Lentapedia')]
[('object', 'http://www.wikidata.org/entity/Q179808'), ('name', "Palme d'Or")]
[('object', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'star on Hollywood Walk of Fam

In [61]:
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
    # bind something
    wd:Q188473 ?p ?object .
    # get the label
    ?p sc:name ?name.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3569'), ('name', 'Cultureel Woordenboek ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('

In [62]:
# Using subClass P279 with action film Q188473

queryString = """
SELECT DISTINCT ?object ?name 
WHERE {
    # bind something
    wd:Q188473 wdt:P279 ?object .
    # get the label
    ?object sc:name ?name.
    FILTER(REGEX(?name,"film"))
    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [63]:
# write your final query
# Retrived BGP for film

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q11424 ?object ?o.
   # get the label
   ?object sc:name ?name.
  
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('object', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('object', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('object', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('object', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('object', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('object', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('object', 'http://www.wikidata.org/prop/di

## Task 2
Identify the BGP for director

In [59]:
# write your queries
# cHECK the details of Director node on Q3455803 FROM THE previous Result

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q3455803 ?p ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q5870903'), ('name', 'Category:Directors')]
[('object', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('object', 'http://www.wikidata.org/entity/Q483501'), ('name', 'artist')]
[('object', 'http://www.wikidata.org/entity/Q2137554'), ('name', 'direction')]
[('object', 'http://www.wikidata.org/entity/Q2462658'), ('name', 'manager')]
[('object', 'http://www.wikidata.org/entity/Q2500638'), ('name', 'creator')]
[('object', 'http://www.wikidata.org/entity/Q2607241'), ('name', 'mise en scène')]
[('object', 'http://www.wikidata.org/entity/Q8104450'), ('name', 'Category:Wikipedia categories named after filmmakers')]
[('object', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]
[('object', 'http://www.wikidata.org/entity/Q10809861'), ('name', 'Template:Infobox film director')]
10


In [83]:
# write your queries
# cHECK the details of Director node on Q3455803 FROM THE previous Result
# [('object', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]


queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q971865 wdt:P57 ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q25089'), ('name', 'Woody Allen')]
[('object', 'http://www.wikidata.org/entity/Q27351'), ('name', 'Senkichi Taniguchi')]
2


In [91]:
# write your queries
# cHECK the details of Director node on Q3455803 FROM THE previous Result
# [('object', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]
# We just need director to search for BGP so 

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q971865 ?object wd:Q27351;
   ?p wd:Q25089.
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]
1


In [92]:
# write your queries
# cHECK the details of Director node on Q3455803 FROM THE previous Result
# [('object', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]
# We just need director to search for BGP

# we Have Retrivied BGP for Director



queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q971865 ?object ?o.
   # get the label
   ?object sc:name ?name.
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('object', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('object', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('object', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('object', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('object', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('object', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('object', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('object', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('object', 'http://www.wikida

Final query for this task

## Task 3
Identify the BGP for film genre

In [93]:
# write your queries

# [('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'director')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q25089 ?p ?object .
   ?p sc:name ?pName.
   ?p sc:name ?name
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [95]:
# write your queries

# Now I will try to look All the objects of Q25089
queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q25089 ?p ?object .
   ?p sc:name ?pName.
   ?object sc:name ?name
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q349357'), ('name', 'Woody Herman')]
[('object', 'http://www.wikidata.org/entity/Q1140914'), ('name', 'Soon-Yi Previn')]
[('object', 'http://www.wikidata.org/entity/Q22673707'), ('name', 'Bechet Dumaine Allen')]
[('object', 'http://www.wikidata.org/entity/Q22673708'), ('name', 'Manzie Tio Allen')]
[('object', 'http://www.wikidata.org/entity/Q18426'), ('name', 'The Bronx')]
[('object', 'http://www.wikidata.org/entity/Q102642'), ('name', 'Diane Keaton')]
[('object', 'http://www.wikidata.org/entity/Q3404523'), ('name', 'David di Donatello for Best Foreign Actor')]
[('object', 'http://www.wikidata.org/entity/Q599316'), ('name', 'New School')]
[('object', 'http://www.wikidata.org/entity/Q19961034'), ('name', 'Woody')]
[('object', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('object', 'http://www.wikidata.org/entity/Q3831197'), ('name', 'Letty Aronson')]
[('object', 'http://www.wikidata.org/entity/Q31579'), ('

In [101]:
# write your queries

# Now I will try to look All the objects of Q971865
# [('object', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]

queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q971865 ?p ?object .
   ?object sc:name ?name
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q25089'), ('name', 'Woody Allen')]
[('object', 'http://www.wikidata.org/entity/Q465224'), ('name', 'American International Pictures')]
[('object', 'http://www.wikidata.org/entity/Q376138'), ('name', 'Charles H. Joffe')]
[('object', 'http://www.wikidata.org/entity/Q6381680'), ('name', 'Kazuo Yamada')]
[('object', 'http://www.wikidata.org/entity/Q1053608'), ('name', "The Lovin' Spoonful")]
[('object', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('object', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('object', 'http://www.wikidata.org/entity/Q1860'), ('name', 'English')]
[('object', 'http://www.wikidata.org/entity/Q22006653'), ('name', 'color')]
[('object', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]
[('object', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('object', 'http://www.wikidata.org/entity/Q723685'), ('name', 'video on demand')]
[('object', 'h

In [102]:
# write your queries

# Now I will try to look All the objects of Q971865
# [('object', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {

   wd:Q971865 ?p ?object .
   ?p sc:name ?name
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('name', 'Netflix ID')]
[('p', '

In [103]:
# write your queries

# Now I will try to look All the objects of Q971865
# [('object', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]


queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q971865 wdt:P136 ?object .
   ?object sc:name ?name
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]
1


In [105]:
# write your queries

# Now I will try to look All the objects of Q971865
# [('object', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
# [('object', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]



queryString = """
SELECT DISTINCT ?object ?name
WHERE {

   wd:Q971865 wdt:P136 ?genre .
   ?genre wdt:P31 ?object.
   ?object sc:name ?name
   
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


Final query for this task

In [107]:
# write your final query

# write your queries

# Now I will try to look All the objects of Q971865
# [('object', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
# [('object', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]

# BGP FOR  Genre movie

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q201658 ?object ?o.
   # get the label
   ?object sc:name ?name.
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('object', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('object', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('object', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('object', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('object', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('object', 'http://www.wikidata.org/prop/direct/P3235'), ('name', 'PhilPapers topic')]
[('object', 'http://www.wikidata.org/prop/direct/P3553'), ('name', 'Zhihu topic ID')]
[('object', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('object', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('object', 'http://www.wikidata.org/

## Task 4
Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

In [117]:
# write your queries

queryString = """
SELECT DISTINCT ?films ?Sname ?country ?name
WHERE {
   # bind something
   ?films wdt:P31 wd:Q5;
       wdt:P136 wd:Q172980;
    wdt:P27 ?country.
    
    ?country wdt:P31 wd:Q6256;
    wdt:P30 wd:Q46.
   # get the label
   ?films sc:name ?Sname.
   ?country sc:name ?name
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('films', 'http://www.wikidata.org/entity/Q309240'), ('Sname', 'Jean Giraud'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
1


Final query for this task

In [118]:
# write your final query

# write your queries

queryString = """
SELECT DISTINCT ?films ?Sname ?country ?name
WHERE {
   # bind something
   ?films wdt:P31 wd:Q5;
       wdt:P136 wd:Q172980;
    wdt:P27 ?country.
    
    ?country wdt:P31 wd:Q6256;
    wdt:P30 wd:Q46.
   # get the label
   ?films sc:name ?Sname.
   ?country sc:name ?name
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)


Results
[('films', 'http://www.wikidata.org/entity/Q309240'), ('Sname', 'Jean Giraud'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
1


## Task 5
Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films).

In [120]:
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
queryString = """
SELECT DISTINCT ?film ?filmname ?year
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
   wdt:P31 wd:Q11424;
   wdt:P577 ?year.
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmname', 'The Concert for New York City'), ('year', '2001-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('year', '2011-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('year', '2011-08-18T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('year', '1990-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('year', '1991-02-28T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmname', "What's Up, Tiger Lily?"), ('year', '1966-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmname', 'Bullets Over Broadway'), ('year', '1994-09-16T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmname', 'Bullets Over Broadway'), ('year', '1995-04-20T00:00:00Z')]
[('film', 'h

In [123]:
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATIN 

queryString = """
SELECT DISTINCT ?film ?filmname (YEAR(?date) AS ?publicationYear)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
   wdt:P31 wd:Q11424;
   wdt:P577 ?date.
   
   filter NOT EXISTS{
   ?film wdt:P577 ?secondDate.
   filter(?secondDate < ?date).
   }
   
   
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmname', 'The Concert for New York City'), ('publicationYear', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('publicationYear', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('publicationYear', '1990')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmname', "What's Up, Tiger Lily?"), ('publicationYear', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmname', 'Bullets Over Broadway'), ('publicationYear', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmname', 'Mighty Aphrodite'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('filmname', 'Broadway Danny Rose'), ('publicationYear', '1984')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('filmname', 'Match Point'), ('publicationYear', '2005')]
[('film', 'http://www.wikidata.org/entity/Q740143

In [124]:
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?film ?filmname (YEAR(?date) AS ?publicationYear)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
   wdt:P31 wd:Q11424;
   wdt:P577 ?date.
   
     
   
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmname', 'The Concert for New York City'), ('publicationYear', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('publicationYear', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('publicationYear', '1990')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('publicationYear', '1991')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmname', "What's Up, Tiger Lily?"), ('publicationYear', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmname', 'Bullets Over Broadway'), ('publicationYear', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmname', 'Bullets Over Broadway'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmname', 'Mighty Aphrodite'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109')

In [131]:
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?film ?filmname (YEAR(?date) AS ?publicationYear) (count(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
   wdt:P31 wd:Q11424;
   wdt:P577 ?date.
   
 
   filter NOT EXISTS{
   ?film wdt:P577 ?secondDate.
   filter(?secondDate < ?date).
   }
   
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('publicationYear', '2011'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('filmname', 'Bananas'), ('publicationYear', '1971'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q187423'), ('filmname', 'Zelig'), ('publicationYear', '1983'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmname', 'The Concert for New York City'), ('publicationYear', '2001'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('publicationYear', '1990'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q614009'), ('filmname', 'Anything Else'), ('publicationYear', '2003'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q910523'), ('filmname', 'Shadows and Fog'), ('publicationYear', '1991'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q1133277'), ('filmname', 

In [127]:
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?film ?filmname (YEAR(?date) AS ?publicationYear) (count(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
   wdt:P31 wd:Q11424;
   wdt:P577 ?date.
   
 
   filter NOT EXISTS{
   ?film wdt:P577 ?secondDate.
   filter(?secondDate < ?date).
   }
   
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmname', 'Midnight in Paris'), ('publicationYear', '2011'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('filmname', 'Bananas'), ('publicationYear', '1971'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q187423'), ('filmname', 'Zelig'), ('publicationYear', '1983'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmname', 'The Concert for New York City'), ('publicationYear', '2001'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmname', 'Alice'), ('publicationYear', '1990'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q614009'), ('filmname', 'Anything Else'), ('publicationYear', '2003'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q910523'), ('filmname', 'Shadows and Fog'), ('publicationYear', '1991'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q1133277'), ('filmname', 

Final query for this task

In [133]:
# write your final query
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?dir ?dirName (YEAR(?date) AS ?publicationYear) (count(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 dir;
   wdt:P31 wd:Q11424;
   wdt:P577 ?date.
   
   ?dir sc:name ?dirName
   FILTER(?dir=wd:Q25089 || ?dir=wd:Q3772)
   
   
 
   filter NOT EXISTS{
   ?film wdt:P577 ?secondDate.
   filter(?secondDate < ?date).
   }
   
   # get the label
   ?film sc:name ?filmname.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
The operation failed QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 13: syntax error at 'dir' before ';'\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-4ddda0dbce-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?dir ?dirName (YEAR(?date) AS ?publicationYear) (count(?film) AS ?numFilms)\nWHERE {\n   # bind something\n   ?film wdt:P57 dir;\n   wdt:P31 wd:Q11424;\n   wdt:P577 ?date.\n   \n   ?dir sc:name ?dirName\n   FILTER(?dir=wd:Q25089 || ?dir=wd:Q3772)\n   \n   \n \n   filter NOT EXISTS{\n   ?film wdt:P577 ?secondDate.\n   filter(?secondDate < ?date).\n   }\n   \n   # get the label\n   ?film sc:name ?filmname.\n    \n}\nLIMIT 50\n"


## Task 6
Identify the BGP for Academy Awards

In [135]:
# write your queries

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
    #BIND SOMETHING
    wd:Q3772 ?p ?object.
    # get the label
    ?object sc:name ?name
    
    
}

LIMIT 100
"""

print("Result")
x=run_query(queryString)

Result
[('object', 'http://www.wikidata.org/entity/Q212026'), ('name', 'Mira Sorvino')]
[('object', 'http://www.wikidata.org/entity/Q2566554'), ('name', '48th British Academy Film Awards')]
[('object', 'http://www.wikidata.org/entity/Q833154'), ('name', 'Edgar Awards')]
[('object', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('object', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('object', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('object', 'http://www.wikidata.org/entity/Q113068'), ('name', 'Hungarian Order of Merit')]
[('object', 'http://www.wikidata.org/entity/Q13452531'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('object', 'http://www.wikidata.org/entity/Q17290934'), ('name', 'Lentapedia')]
[('object', 'http://www.wikidata.org/entity/Q179808'), ('name', "Palme d'Or")]
[('object', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'star on Hollywood Walk of Fame

In [136]:
# write your queries

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
    #BIND SOMETHING
    wd:Q103360 ?p ?object.
    # get the label
    ?object sc:name ?name
    
    
}

LIMIT 100
"""

print("Result")
x=run_query(queryString)

Result
[('object', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('object', 'http://www.wikidata.org/entity/Q160726'), ('name', 'Ang Lee')]
[('object', 'http://www.wikidata.org/entity/Q55215'), ('name', 'Alejandro González Iñárritu')]
[('object', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('object', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('object', 'http://www.wikidata.org/entity/Q28028'), ('name', 'Alfonso Cuarón')]
[('object', 'http://www.wikidata.org/entity/Q212329'), ('name', 'Academy of Motion Picture Arts and Sciences')]
[('object', 'http://www.wikidata.org/entity/Q7146100'), ('name', 'Category:Best Directing Academy Award winners')]
[('object', 'http://www.wikidata.org/entity/Q6667698'), ('name', 'Template:Academy Award for Best Director')]
[('object', 'http://www.wikidata.org/entity/Q495980'), ('name', 'Bong Joon-ho')]
10


Final query for this task

In [137]:
# write your final query

# write your queries

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
    #BIND SOMETHING
    wd:Q103360 ?p ?object.
    # get the label
    ?object sc:name ?name
    filter REGEX (?name, "Academy Awards")
    
    
}

LIMIT 100
"""

print("Result")
x=run_query(queryString)

Result
[('object', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


## Task 7
Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

In [142]:
# write your queries

# write your final query
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?director ?dirName ?film ?filmName ?academnyAward ?academyAwardName
WHERE {
   # bind something
   ?academnyAward wdt:P31 wd:Q19020.
   ?film wdt:P31 wd:Q11424;
   wdt:P57 ?director;
   wdt:P166 ?academnyAward.
   
   ?director sc:name ?dirName.
   ?film sc:name ?filmName.
   ?academnyAward sc:name ?academyAwardName.
   FILTER (?director=wd:Q25089 || ?director=wd:Q3772)
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('academnyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('academnyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('academnyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('dirNam

In [143]:
# write your queries

# write your final query
# write your queries

# Director Number P57
# FILM Q11424
# WOODEY ALLEN Q25089
# SEARCH ALL THE FLIMS WHOSE IS DIRECTED BY WOODEY ALLEN
# LETS FIND PUBLICATON YEAR
# FIND PUBLICATION YEAR OF EACH FILIM

# tRY TO FIND WITH FIRST DATE OF PUBLICATion

queryString = """
SELECT DISTINCT ?director ?dirName ?film ?filmName ?academnyAward ?academyAwardName
WHERE {
   # bind something
   ?academnyAward wdt:P31 wd:Q19020.
   ?film wdt:P31 wd:Q11424;
   wdt:P57 ?director;
   wdt:P166 ?academnyAward.
   
   ?director sc:name ?dirName.
   ?film sc:name ?filmName.
   ?academnyAward sc:name ?academyAwardName.
   FILTER (?director=wd:Q25089 || ?director=wd:Q3772)
    
}

GROUP BY ?film ?filmName ?director ?directorName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmName', 'Django Unchained'), ('academnyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('academnyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmName', 'Annie Hall'), ('academnyAward', 'http://www.wikidata.org/entity/Q102427'), ('academyAwardName', 'Academy Award for Best Picture')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('film'

In [150]:
# write your queries

# write your final query
# write your queries




queryString = """
SELECT DISTINCT ?director ?dirName (MAX(?numAwards) AS ?maxAwards)
WHERE {
SELECT DISTINCT ?director ?dirName ?film ?filmName (COUNT(?academnyAward) AS ?numAwards)
WHERE {
   # bind something
   ?academnyAward wdt:P31 wd:Q19020.
   ?film wdt:P31 wd:Q11424;
   wdt:P57 ?director;
   wdt:P166 ?academnyAward.
   
   ?director sc:name ?dirName.
   ?film sc:name ?filmName.
   ?academnyAward sc:name ?academyAwardName.
   FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
    
}
GROUP BY ?film ?filmName ?director ?directorName
}
GROUP BY ?director ?directorName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
The operation failed QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'Virtuoso 37000 Error SP031: SPARQL compiler: Variable ?dirName is used in the result set outside aggregate and not mentioned in GROUP BY clause\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\n\n##-4ddda0dbce-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT DISTINCT ?director ?dirName (MAX(?numAwards) AS ?maxAwards)\nWHERE {\nSELECT DISTINCT ?director ?dirName ?film ?filmName (COUNT(?academnyAward) AS ?numAwards)\nWHERE {\n   # bind something\n   ?academnyAward wdt:P31 wd:Q19020.\n   ?film wdt:P31 wd:Q11424;\n   wdt:P57 ?director;\n   wdt:P166 ?academnyAward.\n   \n   ?director sc:name ?dirName.\n   ?film sc:name ?filmName.\n   ?academnyAward sc:name ?academyAwardName.\n   FILTER(?director=wd:Q25089 |

Final query for this task

In [151]:
# write your final query

## Task 8
Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query